In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

On importe toutes les feuilles de chacun des fichiers 

In [78]:
# Pour Leroy Merlin
lm_visites = pd.read_excel('DATA Leroy Merlin.xlsx', sheet_name='VISITES ANNUELLES 22 23')
lm_pp = pd.read_excel('DATA Leroy Merlin.xlsx', sheet_name='PP 22 23')
lm_merchandising = pd.read_excel('DATA Leroy Merlin.xlsx', sheet_name='Données brutes MERCH 22 23')
lm_ca_sell = pd.read_excel('DATA Leroy Merlin.xlsx', sheet_name='CA SELL IN LM 22 23')

# Pour Bricomarché
bm_visites = pd.read_excel('DATA Bricomarché.xlsx', sheet_name='VISITES ANNUELLES 22 23')
bm_pp = pd.read_excel('DATA Bricomarché.xlsx', sheet_name='PP 22 23')
bm_merchandising = pd.read_excel('DATA Bricomarché.xlsx', sheet_name='Données brutes MERCH 22 23')
bm_ca_sell = pd.read_excel('DATA Bricomarché.xlsx', sheet_name='Données brutes SELL IN 22 23')

In [79]:
lm_visites['Chaine'] = 'Leroy Merlin'
bm_visites['Chaine'] = 'Bricomarché'

lm_ca_sell['Chaine'] = 'Leroy Merlin'
bm_ca_sell['Chaine'] = 'Bricomarché'

On renomme les colonnes pour qu'elles correspondent entre les datasets


In [80]:
lm_visites.rename(columns={'CODE INTERNE UNIQUE DU POINT DE VENTE': 'Code Point Vente'}, inplace=True)
bm_visites.rename(columns={'CODE INTERNE UNIQUE DU POINT DE VENTE': 'Code Point Vente'}, inplace=True)

lm_ca_sell.rename(columns={'Code du Point de Vente': 'Code Point Vente'}, inplace=True)
bm_ca_sell.rename(columns={'Code du Point de Vente': 'Code Point Vente'}, inplace=True)

### Fusion des datasets

In [81]:
# visites annuelles
visites = pd.concat([lm_visites, bm_visites], ignore_index=True)

# données de CA Sell
ca_sell = pd.concat([lm_ca_sell, bm_ca_sell], ignore_index=True)

In [82]:
# Fusion visites et  ventes
data = pd.merge(visites, ca_sell, on=['Code Point Vente', 'Chaine'], how='inner')

In [83]:
data.head()

,Code Point Vente,Raison Sociale_x,Ville_x,Code Postal_x,Total visites annuelles\n2022,Total visites annuelles\n2023,Chaine,Raison Sociale_y,Code Postal_y,Ville_y,...,Septembre\nEvol,Octobre\n2022,Octobre\n2023,Octobre\nEvol,Novembre\n2022,Novembre\n2023,Novembre\nEvol,Décembre\n2022,Décembre\n2023,Décembre\nEvol
0,20rh8,LEROY MERLIN - 075,ANDELNANS,90400,55,55,Leroy Merlin,LEROY MERLIN - 075,90400,ANDELNANS,...,-0.1830,20311.0,18286.0,-0.0997,16786.0,26561.0,0.5823,20260.0,12641.0,-0.3761
1,20rh8,LEROY MERLIN - 075,ANDELNANS,90400,55,55,Leroy Merlin,LEROY MERLIN - 075,90400,ANDELNANS,...,-0.1830,20311.0,18286.0,-0.0997,16786.0,26561.0,0.5823,20260.0,12641.0,-0.3761
2,20rh8,LEROY MERLIN - 075,ANDELNANS,90400,55,55,Leroy Merlin,LEROY MERLIN - 075,90400,ANDELNANS,...,-0.1517,16087.0,13654.0,-0.1512,11019.0,18512.0,0.6799,16035.0,10299.0,-0.3577
3,20rh8,LEROY MERLIN - 075,ANDELNANS,90400,55,55,Leroy Merlin,LEROY MERLIN - 075,90400,ANDELNANS,...,-0.0510,1077.0,155.0,-0.8562,1042.0,4405.0,3.2285,822.0,651.0,-0.2085
4,20rh8,LEROY MERLIN - 075,ANDELNANS,90400,55,55,Leroy Merlin,LEROY MERLIN - 075,90400,ANDELNANS,...,0.2306,1832.0,3810.0,1.0801,1560.0,536.0,-0.6562,1280.0,762.0,-0.4048


In [84]:
data[['Ville_x', 'Ville_y', 'Code Postal_x', 'Code Postal_y']].head()

# On va donc supprimer les colonnes Ville_y et Code Postal_y car elles sont identiques à Ville_x et Code Postal_x

,Ville_x,Ville_y,Code Postal_x,Code Postal_y
0,ANDELNANS,ANDELNANS,90400,90400
1,ANDELNANS,ANDELNANS,90400,90400
2,ANDELNANS,ANDELNANS,90400,90400
3,ANDELNANS,ANDELNANS,90400,90400
4,ANDELNANS,ANDELNANS,90400,90400


In [85]:
data.drop(['Ville_y', 'Code Postal_y'], axis=1, inplace=True)

data.rename(columns={'Ville_x': 'Ville', 'Code Postal_x': 'Code Postal'}, inplace=True)

In [86]:
codes_postaux = pd.read_csv('code_postaux.csv', sep=',')

# Séparer les coordonnées GPS en Latitude et Longitude
codes_postaux[['Latitude', 'Longitude']] = codes_postaux['coordonnées_gps'].str.split(',', expand=True).astype(float)


In [87]:
codes_postaux.rename(columns={
    'code_commune_insee': 'Code INSEE',
    'nom_de_la_commune': 'Ville',
    'code_postal': 'Code Postal',
    'coordonnées_gps': 'Coordonnees GPS'
}, inplace=True)

In [88]:
codes_postaux

,Code INSEE,Ville,Code Postal,libelle_d_acheminement,ligne_5,Coordonnees GPS,Latitude,Longitude
0,01001,L ABERGEMENT CLEMENCIAT,1400,L ABERGEMENT CLEMENCIAT,NaN,"46.1517018,4.9306005",46.151702,4.930600
1,01002,L ABERGEMENT DE VAREY,1640,L ABERGEMENT DE VAREY,NaN,"46.007131,5.4246442",46.007131,5.424644
2,01004,AMBERIEU EN BUGEY,1500,AMBERIEU EN BUGEY,NaN,"45.9574707,5.37056825",45.957471,5.370568
3,01005,AMBERIEUX EN DOMBES,1330,AMBERIEUX EN DOMBES,NaN,"45.99922935,4.9118718",45.999229,4.911872
4,01006,AMBLEON,1300,AMBLEON,NaN,"45.74831435,5.5927847",45.748314,5.592785
...,...,...,...,...,...,...,...,...
39187,98831,VOH,98883,OUACO,VOH,"-20.878791049401627,164.7404509180226",-20.878791,164.740451
39188,98832,YATE,98834,YATE,NaN,"-22.099978606359787,166.73090135672868",-22.099979,166.730901
39189,98833,KOUAOUA,98818,KOUAOUA,NaN,"-21.458440064574468,165.76614982951872",-21.458440,165.766150
39190,98901,ILE DE CLIPPERTON,98799,ILE DE CLIPPERTON,NaN,NaN,NaN,NaN


In [89]:
data

,Code Point Vente,Raison Sociale_x,Ville,Code Postal,Total visites annuelles\n2022,Total visites annuelles\n2023,Chaine,Raison Sociale_y,Code Chef de Secteur Henkel,Total\nFamille,...,Septembre\nEvol,Octobre\n2022,Octobre\n2023,Octobre\nEvol,Novembre\n2022,Novembre\n2023,Novembre\nEvol,Décembre\n2022,Décembre\n2023,Décembre\nEvol
0,20rh8,LEROY MERLIN - 075,ANDELNANS,90400,55,55,Leroy Merlin,LEROY MERLIN - 075,323,TOTAL HENKEL,...,-0.1830,20311.0,18286.0,-0.0997,16786.0,26561.0,0.5823,20260.0,12641.0,-0.3761
1,20rh8,LEROY MERLIN - 075,ANDELNANS,90400,55,55,Leroy Merlin,LEROY MERLIN - 075,323,NaN,...,-0.1830,20311.0,18286.0,-0.0997,16786.0,26561.0,0.5823,20260.0,12641.0,-0.3761
2,20rh8,LEROY MERLIN - 075,ANDELNANS,90400,55,55,Leroy Merlin,LEROY MERLIN - 075,323,NaN,...,-0.1517,16087.0,13654.0,-0.1512,11019.0,18512.0,0.6799,16035.0,10299.0,-0.3577
3,20rh8,LEROY MERLIN - 075,ANDELNANS,90400,55,55,Leroy Merlin,LEROY MERLIN - 075,323,NaN,...,-0.0510,1077.0,155.0,-0.8562,1042.0,4405.0,3.2285,822.0,651.0,-0.2085
4,20rh8,LEROY MERLIN - 075,ANDELNANS,90400,55,55,Leroy Merlin,LEROY MERLIN - 075,323,NaN,...,0.2306,1832.0,3810.0,1.0801,1560.0,536.0,-0.6562,1280.0,762.0,-0.4048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8016,20pna,BRICOMARCHE URBAIN - B310,LILLE,59000,7,7,Bricomarché,BRICOMARCHE - B310,521,NaN,...,-0.6522,124.0,NaN,NaN,NaN,86.0,NaN,170.0,183.0,0.0764
8017,20pna,BRICOMARCHE URBAIN - B310,LILLE,59000,7,7,Bricomarché,BRICOMARCHE - B310,521,NaN,...,NaN,136.0,109.0,-0.1985,43.0,21.0,-0.5057,18.0,49.0,1.7269
8018,20pna,BRICOMARCHE URBAIN - B310,LILLE,59000,7,7,Bricomarché,BRICOMARCHE - B310,521,NaN,...,NaN,NaN,NaN,NaN,NaN,56.0,NaN,NaN,NaN,NaN
8019,20pna,BRICOMARCHE URBAIN - B310,LILLE,59000,7,7,Bricomarché,BRICOMARCHE - B310,521,NaN,...,NaN,NaN,NaN,NaN,61.0,NaN,NaN,NaN,NaN,NaN


In [90]:
data['Code Postal'] = data['Code Postal'].astype(str)
data['Ville'] = data['Ville'].astype(str).str.upper().str.strip()

# Données des codes postaux
codes_postaux['Code Postal'] = codes_postaux['Code Postal'].astype(str)
codes_postaux['Ville'] = codes_postaux['Ville'].astype(str).str.upper().str.strip()

In [91]:
# Fusionner les données sur 'Code Postal' et 'Ville'
data = pd.merge(data, codes_postaux, on=['Code Postal', 'Ville'], how='left')

# Vérifier les lignes sans coordonnées
missing_coords = data[data['Latitude'].isna() | data['Longitude'].isna()]
print(f"Nombre de lignes sans coordonnées : {len(missing_coords)}")

Nombre de lignes sans coordonnées : 1586


In [97]:
import folium

# Initialiser la carte centrée sur la France
m = folium.Map(location=[46.2276, 2.2137], zoom_start=6)

# Définir des couleurs pour chaque chaîne
chaine_colors = {'Leroy Merlin': 'green', 'Bricomarché': 'red'}

# Ajouter les points sur la carte
for idx, row in data.dropna(subset=['Latitude', 'Longitude']).iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,
        popup=f"{row['Raison Sociale']} ({row['Chaine']})",
        color=chaine_colors.get(row['Chaine'], 'gray'),
        fill=True,
        fill_color=chaine_colors.get(row['Chaine'], 'gray')
    ).add_to(m)

# Enregistrer la carte
m.save('carte_magasins.html')